In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append(r'C:\Users\ASUS\Desktop\repositories\job_recommender')

In [5]:
import pickle
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from networkx.algorithms.link_analysis.pagerank_alg import pagerank


from recommender.core.network_builder import *
from recommender.core import constants 
from recommender.core.latent_semantic_analysis import *
from recommender.core import job_recommender
# from recommender.core import network_builder

In [3]:
companies_df = pd.read_csv(r'C:\Users\ASUS\Desktop\job_recommender\data\companies.csv')
jobs_df = pd.read_csv(r'C:\Users\ASUS\Desktop\job_recommender\data\jobs.csv')
cv = pd.read_csv(r'C:\Users\ASUS\Desktop\job_recommender\data\cvdata\ResumeDataSet.csv', encoding = 'utf-8')

In [39]:
# transform companies_df and jobs_df into dicts
employers_data = {}
jobs_data = {}
for i, row in companies_df.iterrows():
    employer_data = {'company_name': row['company_name'],
                   'average_rating': row['average_rating'],
                   'num_review': row['num_review'],
                   'city': row['city'],
                   'type': row['type'],
                   'num_employee': row['num_employee'],
                   'country': row['country'],
                   'working_day': row['working_day'],
                   'OT': row['OT'],
                   'overview': row['overview'],
                   'expertise': row['expertise'],
                   'benifit': row['benifit'],
                   'logo_link': row['logo_link']}
    
    employers_data[row['company_id']] = employer_data
    
for i, row in jobs_df.iterrows():
    job_data = {'company_id': row['company_id'],
               'job_name': row['job_name'],
               'taglist': row['taglist'],
               'location': row['location'], 
               'three_reasons': row['three_reasons'],
               'description': row['description']}
    jobs_data[row['job_id']] = job_data
    

In [68]:
G = nx.MultiDiGraph(name = 'Jobs graph')
# first add all employer nodes to the network and its data
for employer_id, employer_data in employers_data.items():
    G.add_node(employer_id, node_type = 'employer', **employer_data)
    
# add all job nodes and the bidirectional edges from job node to employer node
for job_id, job_data in jobs_data.items():
    G.add_node(job_id, node_type = 'job', **job_data)
    # add two edges between job and its employers\
    G.add_edge(job_id, job_data['company_id'], weight = 1)
    G.add_edge(job_data['company_id'], job_id, weight = 1)

In [74]:
G['kms-technology']

AdjacencyView({'kms-technology:jrsr_qa_engineer_kms_labs_bonus': {0: {'weight': 1}}, 'kms-technology:engineering_manager_bonus': {0: {'weight': 1}}, 'kms-technology:fullstack_mobile_mobilenodejs_kobiton': {0: {'weight': 1}}, 'kms-technology:jrsrprincipal_java_developer_bonus': {0: {'weight': 1}}, 'kms-technology:product_manager_kms_labs_bonus': {0: {'weight': 1}}, 'kms-technology:sr_it_business_analyst_english_bonus': {0: {'weight': 1}}, 'kms-technology:fullstack_dev_reactjsnodejs_kobiton': {0: {'weight': 1}}, 'kms-technology:senior_ruby_on_rails_engineer_bonus': {0: {'weight': 1}}, 'kms-technology:senior_data_engineer_bonus': {0: {'weight': 1}}, 'kms-technology:srjr_fullstack_nodejsreactjs_bonus': {0: {'weight': 1}}, 'kms-technology:juniorsenior_test_engineer_bonus': {0: {'weight': 1}}, 'kms-technology:jrsrprincipal_net_developer_bonus': {0: {'weight': 1}}})

In [3]:
nb = network_builder.NetworkBuilder(constants.EMPLOYERS_DATA_PATH,
                                    constants.JOBS_DATA_PATH, constants.CV_DATAPATH )

In [5]:
nb.build()

2021-11-26 06:47:00,391 - Start building the master Graph...
2021-11-26 06:47:00,418 - Master Graph is built.
2021-11-26 06:47:00,419 - Creating LSA comparer
2021-11-26 06:47:00,420 - Loading LSA comparer from /home/huynhhao/Desktop/job_recommender/data/network_data/lsa.pkl
2021-11-26 06:47:00,518 - Adding relations edges...


In [8]:
len(nb.G.nodes)

4414

In [31]:
nb.lsa.preprocess_text(nb.G.nodes['kms-technology']['overview'])

'established km technology usbased engineering service company development center vietnam km technology trusted international client superior quality product expertise vietnamese engineer km build successfully launch software company internal startup _unknown_ km lab _unknown_ company include qasymphony kobiton katalon _unknown_ brand km solution company serf asia pacific region offering technology solution consulting bringing advanced latest technology asian market km committed making _unknown_ social impact partnering _unknown_ organization give _unknown_ student fair better employment opportunity company regularly _unknown_ university participates training activity specialized event sponsor scholarship program vietnam company received many prestigious industry award named one greatest place work vietnam asia u many year row'

In [6]:
path = r'C:\Users\ASUS\Desktop\repositories\job_recommender\data\network_data\network_builder.pkl'
with open(path, 'rb') as f:
    network = pickle.load(f)

ModuleNotFoundError: No module named 'latent_semantic_analysis'

In [4]:
jrec = job_recommender.JobRecommender(network.G, network.lsa)

In [5]:
new_guy = {'expertise': 'Python Developer', 
          'resume': "eeking to increase data efficiency for Contranix Capital Inc. Achievements include creating data regression models to predict company stock prices with 25% more accuracy than historical average. Achieved 20% improvement in investment returns across all clients. Highly skilled in machine learning, data visualization, and creative thinking."}

In [6]:
jrec.add_node_to_graph('candidate', new_guy)

In [7]:
jrec.rank_nodes(True, jrec.target_node, 'job')

{'salto-vietnam:qctest_engineer_manualautomation': 7.18349189970594e-06,
 'hello-health-group:data_engineer': 5.6938845810330755e-06,
 'hdv-mobisoft:tester_qa_qc_business_analyst': 4.373642333942626e-06,
 'incepit:java_dev_javascript_mysql': 3.8461185564649104e-06,
 'exidea-vietnam:senior_php_backend_dev_laravel_mysql': 3.5243743311143412e-06,
 'virtual-internships:uiux_designer_webapp_hcmcremote': 3.3443584877578162e-06,
 'ipfs:manual_tester_qa_qc': 3.2534026665175915e-06,
 'inmeeting-jsc:software_developer_net_mysql_mvc': 3.05643784768139e-06,
 'go-game-vietnam-limited-liability-company:unity_games_developer': 2.948183735307761e-06,
 'de-heus-asia:sr_software_developer_netcsqloop': 2.915852510290123e-06,
 'fwd:senior_business_analyst': 2.7538775953048787e-06,
 'bluebottle-digital-viet-nam:tester_qa_qc_english_upto_1200': 2.687537845052765e-06,
 'simple-tech-investment:data_analytics_manager': 2.608711012269728e-06,
 'hahahashopping:php_developer_laravel_nodejs': 2.5457481980566468e-0

In [21]:
results = jrec.search('data analysis')

In [10]:
jrec._rank_node_with_context(jrec.target_node, results, 'job')

{'cinnamon-ai-labs:it_support_engineer_hcm': 0.024974056649070965,
 'cinnamon-ai-labs:engineering_manager': 0.024974056649070965,
 'sentifi-vietnam:software_engineer_databasejavapython': 0.024266922210498032,
 'cinnamon-ai-labs:frontend_engineer_signon_bonus': 0.023130976371113634,
 'tiki-corporation:senior_qc_engineer_tester': 0.021836384584945728,
 'asilla:software_engineer_databasejavapython': 0.02164962488405458,
 'pascalia-asia-vietnam:phpvuejsnodejsreactjs': 0.02130450637909599,
 'viettel-group:30_java_engineer_spring_mvc_upto_60tr': 0.02115982005113183,
 'island-peak-cloud:devops_engineer_pythondocker': 0.020507195218184546,
 'younet:qc_engineer_middle_senior': 0.020434537972202717,
 'unifiedpost:data_engineer_pythongcp': 0.020426451411077944,
 'asilla:software_engineer_python_linux_1500': 0.020363605090140808,
 'mobiletech:ai_engineer_text_processing': 0.02033328605782751,
 'viettel-group:10_bigdata_engineers_java_scala_sql': 0.020125039778458496,
 'katalon:lead_software_engine

In [22]:
results

['avenue',
 'geocomply-geotech-hub-vietnam',
 'digihubs-jsc',
 'song-technologies-llc',
 'sentifi-vietnam',
 'k-system-and-solutions',
 'kompa',
 'teamscal',
 'ogilvy',
 'creatory',
 'grevo',
 'cong-ty-co-phan-citics',
 'bizone',
 'otanics-technology',
 'kms-technology:product_manager_kms_labs_bonus',
 'kms-technology:senior_data_engineer_bonus',
 'nab-in-collaboration-with-positive-thinking-company-in-vietnam:principle_data_engineer_mediorsenior',
 'gft-technologies-vietnam:python_developer_data_engineerairflow',
 'tyme:qasenior_manual_engineer_tester_eng',
 'hansen-technologies:product_owner_business_analystenglish',
 'dek-technologies:data_engineer_java_python_scala',
 'titan-technology-corporation:system_engineer',
 'm_service-momo:product_owner_business_analyst_uiux',
 'nvg-technology:business_intelligence_engineer_bi_sql',
 'nvg-technology:software_development_engineer_in_test',
 'vng-corporation:senior_data_engineer_zalopay',
 'fpt-software:technical_business_analyst',
 'capgemi